##### Example 1

In [ ]:
def train(): print("training...")

In [ ]:
train()

training...


Write a flow that execute the function `train` using `metaflow`

In [ ]:
from metaflow import FlowSpec, step

In [ ]:
class Training(FlowSpec):
    @step
    def start(self):
        self.next(self.train)
        
    @step
    def train(self):
        train()
        self.next(self.end)
    
    @step
    def end(self):
        print("Evaluating...")

In [ ]:
Training

__main__.Training

##### Example 2

In [ ]:
from metaflow import FlowSpec, step

In [ ]:
class CounterFlow(FlowSpec):
    @step
    def start(self):
        self.count = 0
        self.next(self.add)
    
    @step
    def add(self):
        print("The count is", self.count, "before incrementing")
        self.count += 1
        self.next(self.end)
    
    @step
    def end(self):
        self.count += 1
        print("The final count is", self.count)

##### Example 3

In [ ]:
class Training:
    def train(self):
        secret = 69
        print(f"secret {secret}")

Create a flow similar to `Training`, but allow user to pass the value of `secret` in command line

In [ ]:
from metaflow import FlowSpec, Parameter, step

In [ ]:
class Flow(FlowSpec):
    secret = Parameter(
        "secret",
        required=True # can be ignored
    )

    @step
    def start(self): self.next(self.train)

    @step
    def train(self):
        print(f"secret {self.secret}")
        self.next(self.end)

    @step
    def end(self): pass

In [ ]:
Flow

__main__.Flow

### 3.2 Branching and merging

##### Example 1

In [ ]:
class Training:
    def train(self): print("training...")
    
    def eval(self): print("evaluating...")

Create a flow that similar to `Training` but execute `train` and `eval` tasks in parallel

**Hint**: There's an extra method.

In [ ]:
from metaflow import FlowSpec, step

In [ ]:
class TrainingFlow(FlowSpec):
    @step
    def start(self):
        self.next(self.train, self.eval)

    @step
    def train(self):
        print("training...")
        self.next(self.join)

    @step
    def eval(self):
        print("evaluating...")
        self.next(self.join)

    @step
    def join(self, inputs):
        self.next(self.end)

    @step
    def end(self): pass

In [ ]:
TrainingFlow

__main__.TrainingFlow

##### Example 2

In [ ]:
words = ["persistence", "is", "all", "you", "need"]

In [ ]:
words

['persistence', 'is', 'all', 'you', 'need']

Write a flow that counts the length of each word in the list `["persistence", "is", "all", "you", "need"]` in parallel using a dynamic branch, and returns the longest word

**Hint**: `max(words, key=lambda x: len(x))`

In [ ]:
from metaflow import FlowSpec, step

In [ ]:
class CountFlow(FlowSpec):
    @step
    def start(self):
        self.words = words
        self.next(self.count, foreach="words")

    @step
    def count(self):
        self.word = self.input
        self.length = len(self.input)
        self.next(self.join)

    @step
    def join(self, inputs):
        self.longest = max(
            inputs, key=lambda x: x.length
        ).word
        self.next(self.end)

    @step
    def end(self):
        print(self.longest, "wons!")

In [ ]:
CountFlow

__main__.CountFlow

In [ ]:
from metaflow import Flow